In [2]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from scipy.interpolate import splrep, BSpline
import krotov
import qutip
from qutip import Qobj
import matplotlib.pylab as plt
from scipy.interpolate import splrep, BSpline
from qutip import Qobj
from IPython import display
import ipywidgets as widgets
from IPython.display import display, Javascript, clear_output
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import splrep, BSpline
from ipywidgets import Output, interactive
from scipy.interpolate import splrep, BSpline
import qrcode
from IPython.display import Image

In [3]:
def Give_me_the_dynamics(user_name):

    global highScore

    currentScore = 0
        
    # Backend functions

    def Omega_P2_Guess(t, args):
        """Guess for the imaginary part of the pump pulse"""
        return 0.0

    def Omega_S2_Guess(t, args):
        """Guess for the imaginary part of the Stokes pulse"""
        return 0.0

    def Hamiltonian(Omega_P1_Smooth, Omega_S1_Smooth, E1=0.0, E2=10.0, E3=5.0, Omega_P=9.5, Omega_S=4.5):
        """Lambda-system Hamiltonian in the RWA"""

        # detunings
        P = E1 + Omega_P - E2
        S = E3 + Omega_S - E2

        H_0 = qutip.Qobj([[P, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, S]])

        HP_Re = -0.5 * qutip.Qobj([[0.0, 1.0, 0.0], [1.0, 0.0, 0.0], [0.0, 0.0, 0.0]])
        HP_Im = -0.5 * qutip.Qobj([[0.0, 1.0j, 0.0], [-1.0j, 0.0, 0.0], [0.0, 0.0, 0.0]])

        HS_Re = -0.5 * qutip.Qobj([[0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0]])
        HS_Im = -0.5 * qutip.Qobj([[0.0, 0.0, 0.0], [0.0, 0.0, 1.0j], [0.0, -1.0j, 0.0]])

        return [
            H_0,
            [HP_Re, Omega_P1_Smooth],
            [HP_Im, Omega_P2_Guess],
            [HS_Re, Omega_S1_Smooth],
            [HS_Im, Omega_S2_Guess],
        ]

    def RWA_Target_State(Ket_3, E2=10.0, Omega_S=4.5, T=5):
        return np.exp(1j * (E2 - Omega_S) * T) * Ket_3

    def Plot_Pulses(pulse, tlist, label):
        fig, ax = plt.subplots()
        if callable(pulse):
            pulse = np.array([pulse(t, args=None) for t in tlist])
        ax.plot(tlist, pulse)
        ax.set_xlabel('time')
        ax.set_ylabel('%s pulse amplitude' % label)

    def Plot_Population(result):
        with plot_output3: 
            clear_output(wait=True)
            fig, ax = plt.subplots()
            ax.plot(result.times, result.expect[0], label='State 1')
            ax.plot(result.times, result.expect[1], label='State 2')
            ax.plot(result.times, result.expect[2], label='State 3')
            ax.legend()
            ax.set_xlabel('time')
            ax.set_ylabel('population')
            ax.set_title('State Populations')
            plt.rcParams['lines.linewidth'] = 2
            plt.rcParams['axes.labelsize'] = 18
            plt.rcParams['axes.titlesize'] = 20
            plt.tick_params(axis='both', which='major', labelsize=18)
            plt.show()
            currentScoreLabel = widgets.Label('Current Score: ' + str(currentScore))
            highScoreLabel = widgets.Label('High Score: ' + str(highScore))
            score_layout = widgets.VBox([currentScoreLabel, highScoreLabel])
            display(score_layout)

            submit_button = widgets.Button(
                description='Submit to Scoreboard',
                disabled=False,
                button_style='success',
                tooltip='Click me',
                icon='check'
            )

            submit_button.on_click(submit)

            display(submit_button)

    def submit(ev):
        user_name_final = user_name.replace(' ', '-')
        Stringbuilder = "https://gmscoreboard.com/api/set-score/?tagid=94b47096b96f2864e21376a548822b09&player=" + user_name_final + "&score=" + str(highScore)

        qr = qrcode.make(Stringbuilder)

        # Save the qr code as an image
        qr.save('qr_code.png')

        with image_output:
            clear_output(wait=True)
            # Display the qr code
            image = Image('qr_code.png')
            display(image)

    def Omega_Smooth(Omega_Inputs, T_end):
        T_axis = np.arange(0, T_end + 1)
        tck = splrep(T_axis, Omega_Inputs, s=0)
        t_smooth = np.linspace(0, T_end, 500)
        Pulse_smooth = BSpline(*tck)(t_smooth)
        return Pulse_smooth

    T_end = 11
    
    # Getting Slider Values
    User_input_P = [slider.value for slider in point_sliders1]
    User_input_S = [slider.value for slider in point_sliders2]
    
    User_input_P.insert(0, 0)
    User_input_P.append(0)
    User_input_P.append(0)
    User_input_S.insert(0, 0)
    User_input_S.append(0)
    User_input_S.append(0)
    
    # We smooth the Values
    Omega_P1_Smooth = Omega_Smooth(User_input_P, T_end)
    Omega_S1_Smooth = Omega_Smooth(User_input_S, T_end)

    # We Solve the System
    H = Hamiltonian(Omega_P1_Smooth, Omega_S1_Smooth)

    Ket_1 = qutip.Qobj(np.array([1.0, 0.0, 0.0]))
    Ket_2 = qutip.Qobj(np.array([0.0, 1.0, 0.0]))
    Ket_3 = qutip.Qobj(np.array([0.0, 0.0, 1.0]))

    Proj_1 = qutip.ket2dm(Ket_1)
    Proj_2 = qutip.ket2dm(Ket_2)
    Proj_3 = qutip.ket2dm(Ket_3)

    # Target State Definition
    Psi_Target = RWA_Target_State(Ket_3)

    # Instantiating Control Objective
    Objective = krotov.Objective(initial_state=Ket_1, target=Psi_Target, H=H)

    # Show Pulse Plots
    t = np.linspace(0, T_end, 500)
    #Plot_Pulses(H[1][1], t, 'Ωₚ')
    #Plot_Pulses(H[3][1], t, 'Ωₛ')

    Guess_Dynamics = Objective.mesolve(t, e_ops=[Proj_1, Proj_2, Proj_3])

    currentScore = int(Guess_Dynamics.expect[2][499] * 100)

    if currentScore > int(highScore):
        highScore = currentScore

    Plot_Population(Guess_Dynamics)

def update_plot(point1_1, point2_1, point3_1, point4_1, point5_1, point6_1, point7_1, point8_1, point9_1,
                point1_2, point2_2, point3_2, point4_2, point5_2, point6_2, point7_2, point8_2, point9_2):
    X = np.arange(0, 11)
    Y1 = np.array([0, point1_1, point2_1, point3_1, point4_1, point5_1, point6_1, point7_1, point8_1, point9_1, 0])
    Y2 = np.array([0, point1_2, point2_2, point3_2, point4_2, point5_2, point6_2, point7_2, point8_2, point9_2, 0])

    with plot_output1:
        clear_output(wait=True)
        plt.figure(figsize=(6, 4))
        plt.scatter(X, Y1)
        plt.title('Ωₚ')
        plt.xlabel('Time')
        plt.ylabel('Amplitude')
        plt.ylim(-1.25, 1.25)
        Omega_P1_Smooth = Omega_Smooth(Y1, 10)
        t_smooth = np.linspace(0, 10, 500)
        plt.plot(t_smooth, Omega_P1_Smooth, color='r')
        plt.rcParams['lines.linewidth'] = 2
        plt.rcParams['axes.labelsize'] = 18
        plt.rcParams['axes.titlesize'] = 20
        plt.tick_params(axis='both', which='major', labelsize=18)
        plt.tight_layout()
        plt.show()

    with plot_output2:
        clear_output(wait=True)
        plt.figure(figsize=(6, 4))
        plt.scatter(X, Y2)
        plt.title('Ωₛ')
        plt.xlabel('Time')
        plt.ylabel('Amplitude')
        plt.ylim(-1.25, 1.25)
        Omega_P2_Smooth = Omega_Smooth(Y2, 10)
        t_smooth = np.linspace(0, 10, 500)
        plt.plot(t_smooth, Omega_P2_Smooth, color='r')
        plt.rcParams['lines.linewidth'] = 2
        plt.rcParams['axes.labelsize'] = 18
        plt.rcParams['axes.titlesize'] = 20
        plt.tick_params(axis='both', which='major', labelsize=18)
        plt.tight_layout()
        plt.show()

def Omega_Smooth(Omega_Inputs, T_end):
    # Pre-established Time-Domain
    T_axis = np.linspace(0, T_end, len(Omega_Inputs))

    # Random Values given by User
    tck = splrep(T_axis, Omega_Inputs, s=0)

    # Making the random values into Smooth Function (time below is also pre-established)
    t_smooth = np.linspace(0, T_end, 500)
    Pulse_smooth = BSpline(*tck)(t_smooth)

    return Pulse_smooth

def name_click(ev):
    global user_name
    user_name = name_box.value
    stir_button.disabled = False
    with name_output:
        print("Welcome " + name_box.value)

# Initial values for the points
initial_points1 = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
initial_points2 = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

# Create sliders for each point on both plots
point_sliders1 = [widgets.FloatSlider(value=initial_points1[i], min=-1.0, max=1.0, step=0.01, description=f'Ωₚ(t = {i+1})')
                  for i in range(9)]
point_sliders2 = [widgets.FloatSlider(value=initial_points2[i], min=-1.0, max=1.0, step=0.01, description=f'Ωₛ(t = {i+1})')
                  for i in range(9)]

laser1Label = widgets.Label('Define your Ωₚ pulse')
laser2Label = widgets.Label('Define your Ωₛ pulse')

# Create separate containers for the sliders in two columns
sliders_column1 = widgets.VBox(point_sliders1)
sliders_column2 = widgets.VBox(point_sliders2)

sliders_column11 = widgets.VBox([laser1Label, sliders_column1])
sliders_column22 = widgets.VBox([laser2Label, sliders_column2])

# Create Output widgets for the plots
plot_output1 = Output()
plot_output2 = Output()
plot_output3 = Output()

right_layout = widgets.VBox()
box_layout = widgets.Layout(align_items='stretch')
label_layout = widgets.Layout(width='600px', height='flex', align_items='flex-end')
text = 'Many quantum mechanical systems can be described by a few relevant “states”. For example, an atom in its ground state (“state 1”) can be driven by laser light to an excited state (“state 2”). In this case, all other states of the atom can be neglected. In this game, we look at an atom that is excited by two lasers, one driving the transition between states 1 and 2 (\(Ωₚ\)), and one driving the transition between states 2 and 3 (\(Ωₛ\)). The goal of the game is to help the atom from state 1 to state 3 by choosing “good” laser pulses. Adjust the sliders to design laser pulses of \(Ωₚ\) and \(Ωₛ\). Then, click on “Shoot the laser” to see how the state of the atom evolves in time. The curves in the right-hand plot show the populations in each of the three states. Your score is equal to the “fraction” or probability that the atom makes it to state 3.'
game_description_text = f"""
<div style="text-align: justify;">
    {text}
</div>
"""
image_output = Output()

game_description = widgets.HTML(value=game_description_text, layout=label_layout)
right_layout = widgets.VBox([plot_output3, game_description, image_output])
right_layout.align = "bottom"

# Interactively update the plot based on slider values
interactive_plot = widgets.interactive(update_plot,
                    point1_1=point_sliders1[0],
                    point2_1=point_sliders1[1],
                    point3_1=point_sliders1[2],
                    point4_1=point_sliders1[3],
                    point5_1=point_sliders1[4],
                    point6_1=point_sliders1[5],
                    point7_1=point_sliders1[6],
                    point8_1=point_sliders1[7],
                    point9_1=point_sliders1[8],
                    point1_2=point_sliders2[0],
                    point2_2=point_sliders2[1],
                    point3_2=point_sliders2[2],
                    point4_2=point_sliders2[3],
                    point5_2=point_sliders2[4],
                    point6_2=point_sliders2[5],
                    point7_2=point_sliders2[6],
                    point8_2=point_sliders2[7],
                    point9_2=point_sliders2[8])

# Create a button and its click event handler as before
stir_button = widgets.Button(description="Shoot the Laser!", disabled=True)

name_box = widgets.Text(
    value='Purdue Pete',
    placeholder='Name',
    description='Please Enter Your Name:',
    disabled=False,
    style= {'description_width': 'initial'}
)

name_button = widgets.Button(
    description='Enter',
    disabled=False,
    button_style='info',
    tooltip='Click me',
    icon='check'
)

name_button.on_click(name_click)

info_box = widgets.HBox([name_box, name_button])

name_output = Output()
currentScore = 0
highScore = 0

def run_all(ev):
    global highScore
    Give_me_the_dynamics(user_name)

# Connect the button's click event to the run_all function
stir_button.on_click(run_all)

# Arrange the two slider columns side by side
sliders_layout = widgets.HBox([sliders_column11, sliders_column22])

# Create a VBox to contain the interactive plot, the button, and the plot outputs
input = widgets.VBox([info_box, name_output, sliders_layout, plot_output1, plot_output2])

final = widgets.HBox([input, stir_button, right_layout])

# Display the VBox
display(final)
